Lautaro Silva

Carrera: Lic. en Ciencias Fisicas

DNI: 43441919

LU: 879/21

Email: lautarosilvapizzi@gmail.com

# Exercise!

Consider the equation

$$x^{\prime\prime}(t)+\alpha x^{\prime}(t)-\beta x(t)=0$$

with initial conditions: $x(0)=1$, $x^\prime(0)=0$, and $t \in [0,1]$.

**a)** Solve the equation using the `neurodiffeq` module for the values $\alpha = \beta = 1$. Note: you CAN'T fix $\beta=0$ and use the bundle method only for $\alpha$.

**b)** Compute the percentage difference between the PINN-based solution and the analytical one in the range $[0,1]$. Now do the same for the range $[-1,2]$. 

**Hint:** The analytical solution for these parameter values is:

$$x(t) = \frac{1}{2\sqrt{5}}\left((1+\sqrt{5})e^{(-1+\sqrt{5})t/2}+(-1+\sqrt{5})e^{-(1+\sqrt{5})t/2}\right)$$

**c)** Achieve an accuracy such that the percentage error is less than 1% for all values of t (if your solution is not so good, go back to training!).

**d)** Compute and plot the residuals and try to interpretate them.

Send your solution to gomezlucajavier@gmail.com.

**Note:** You can solve this exercise with or without using the *bundle method*. If you choose to use the bundle method, be careful with the number of inputs to the networks, and also with the definition of `eq_param_index` in the solver. While the bundle method might make it easier to follow the structure of the code, it may also lead to longer training times.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
from neurodiffeq import diff
from neurodiffeq.networks import FCNN
from neurodiffeq.generators import Generator1D
from neurodiffeq.conditions import BundleIVP
from neurodiffeq.solvers import BundleSolver1D
from sklearn.metrics import mean_absolute_error

In [7]:
# Tiempos para ambos rangos
t_0_i, t_f_i = 0.0, 1.0
t_0_ii, t_f_ii = -1.0, 2.0

# Rangos de parametros para alpha y beta (los hago cerca de lo deseado para ahorrar costo computaciona;)
alpha_0, alpha_f = 0.9, 1.1
beta_0, beta_f = 0.9, 1.1

# Condiciones Iniciales
x0, x0_dot = 1.0, 0.0
conds = [BundleIVP(t_0_i, x0), BundleIVP(t_0_i, x0_dot)]

def ODE(x, x_p, t, alpha, beta):
    res1 = diff(x, t) - x_p
    res2 = diff(x_p, t) + alpha * x_p - beta * x
    return [res1, res2]

def loss(res, x, t):
    return (res ** 2).mean()

nets = [FCNN(n_input_units=3, hidden_units=(32,32,32,)) for _ in range(2)] # 3 inputs (t, alpha, beta)
adam = torch.optim.Adam(set([p for net in nets for p in net.parameters()]), lr=1e-3)

batch_size = 128
tg_t = Generator1D(batch_size, t_min=t_0_i, t_max=t_f_i)
vg_t = Generator1D(batch_size, t_min=t_0_i, t_max=t_f_i)

tg_alpha = Generator1D(batch_size, t_min=alpha_0, t_max=alpha_f)
vg_alpha = Generator1D(batch_size, t_min=alpha_0, t_max=alpha_f)

tg_beta = Generator1D(batch_size, t_min=beta_0, t_max=beta_f)
vg_beta = Generator1D(batch_size, t_min=beta_0, t_max=beta_f)

train_gen = tg_t ^ tg_alpha ^ tg_beta
valid_gen = vg_t ^ vg_alpha ^ vg_beta

In [9]:
solver = BundleSolver1D(
    ode_system=ODE,
    conditions=conds,
    t_min=t_0_i, t_max=t_f_i,
    theta_min=(alpha_0, beta_0),
    theta_max=(alpha_f, beta_f),
    eq_param_index=(0, 1),
    nets=nets,
    train_generator=train_gen,
    valid_generator=valid_gen,
    optimizer=adam,
    loss_fn=loss,
)

In [10]:
solver.fit(max_epochs=5000)

Training Progress:   0%|                                                                      | 0/5000 [00:00<…

KeyboardInterrupt: 